In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Reading hgvs4variation.txt

In [ ]:
hgvs_var_wd = pd.read_csv("hgvs4variation.txt", delimiter = '\t')
hgvs_var_wd.iloc[1:4]

### Reading variant_summary.txt

In [ ]:
variant_summ_wd = pd.read_csv('variant_summary.txt', delimiter = '\t')
#variant_summ_wd.head(5)

# DICTIONNARIES

In [ ]:

daatol = {
    "Cys": "C",
    "Asp": "D",
    "Ser": "S",
    "Gln": "Q",
    "Lys": "K",
    "Ile": "I",
    "Pro": "P",
    "Thr": "T",
    "Phe": "F",
    "Asn": "N",
    "Gly": "G",
    "His": "H",
    "Leu": "L",
    "Arg": "R",
    "Trp": "W",
    "Ala": "A",
    "Val": "V",
    "Glu": "E",
    "Tyr": "Y",
    "Met": "M",
}

In [ ]:
import os

Text file with all genes

## Dic with all sequences 

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
from biotite.sequence.io.fasta import FastaFile, get_sequences
seq_dic = {}
infile = FastaFile()
infile.read(os.path.expanduser("~/MIT/clinvar_genes_seq.fasta"))
#header = infile.split('GN=')[1].split(' ')[0]
seqs = get_sequences(infile)
for header, seq in tqdm(seqs.items()):
    #items c'est le bon truc
    gene = header.split('GN=')[1].split(' ')[0]
    seq_dic[gene] = str(seq)

In [ ]:
effective_genes = [g for g in seq_dic]
effective_genes = set(effective_genes)

In [ ]:
gene_names = set([n for n in hgvs_var_wd["Symbol"]  if n != 'TERC' ])
dic = {gene: {} for gene in effective_genes}

## Dic of dic of dic: (Gene) Symbol: {AlleleID : {Protein Change : Clinical Significance}}

In [ ]:
import re 
regex = re.compile('^[A-Za-z]{3}[0-9]+[A-Za-z]{3}$')
unwanted_mut = {'Ter', 'del', 'Del', 'DEL', 'dup', 'TER', 'Xaa'}
for row in hgvs_var_wd.itertuples():
    if (row.Symbol in effective_genes):
    #exclude alleles that do not make a change in protein
        if (row.ProteinChange != '-'):
            proteinChange = row.ProteinChange.split('.')[1]
            #make sure proteinChange matches the format 3 letters + number + 3 letters
            if (regex.match(proteinChange)):
                if (proteinChange[:3] not in unwanted_mut) and (proteinChange[-3:] not in unwanted_mut):
                    loc = int(proteinChange[3:-3])
                    #19/08 added the condition to only keep the genes that are compatible with alphafold
                    #if(loc < 1022):
                    if(len(seq_dic[row.Symbol]) > loc-1):
                        wt = proteinChange[:3]
                        wt = wt[0].upper() + wt[1:].lower()
                        #voir si on garde ça exactement ou si on change cette condition
                        if(wt in daatol and daatol[wt] == seq_dic[row.Symbol][loc-1]):
                                dic[row.Symbol][row.AlleleID] = {"Change" : proteinChange}
for row in variant_summ_wd.itertuples():
    if (row.GeneSymbol in dic):
        if (row.AlleleID in dic[row.GeneSymbol]):
            if (row.ClinicalSignificance is not None):
                dic[row.GeneSymbol][row.AlleleID]["ClinicalSignificance"] = row.ClinicalSignificance

In [ ]:
pathogenic = {'Pathogenic','Pathogenic/Likely pathogenic','Pathogenic/Likely pathogenic; risk factor',
                  'Likely pathogenic', 'risk factor',
                  'Likely pathogenic; Affects',
                  'Likely pathogenic; association',
                  'Likely pathogenic; drug response',
                  'Likely pathogenic; other',
                  'Likely pathogenic; risk factor',
                  'Likely risk allele',
                  'Pathogenic/Likely pathogenic; drug response',
                  'Pathogenic/Likely pathogenic; other',
                  'Pathogenic/Likely risk allele',
                  'Pathogenic; Affects',
                  'Pathogenic; association',
                  'Pathogenic; drug response',
                  'Pathogenic; drug response; other',
                  'Pathogenic; other',
                  'Pathogenic; protective',
                  'Pathogenic; risk factor'}
benign = {'Benign', 'Benign/Likely benign', 'Likely benign',
            'Benign/Likely benign; association',
            'Benign/Likely benign; drug response',
            'Benign/Likely benign; drug response; other',
            'Benign/Likely benign; other',
            'Benign/Likely benign; other; risk factor',
            'Benign/Likely benign; risk factor',
            'Benign; Affects',
            'Benign; association',
            'Benign; confers sensitivity',
            'Benign; drug response',
            'Benign; other',
            'Benign; protective',
            'Benign; risk factor',
            'Likely benign; Affects',
            'Likely benign; drug response',
            'Likely benign; drug response; other',
            'Likely benign; other',
            'Likely benign; risk factor'}
vus = {'Uncertain significance', 'Conflicting interpretations of pathogenicity',
          'Affects',
          'Affects; association',
          'Affects; risk factor',
          'Conflicting interpretations of pathogenicity; Affects',
          'Conflicting interpretations of pathogenicity; association',
          'Conflicting interpretations of pathogenicity; association; risk factor',
          'Conflicting interpretations of pathogenicity; drug response',
          'Conflicting interpretations of pathogenicity; other',
          'Conflicting interpretations of pathogenicity; other; risk factor',
          'Conflicting interpretations of pathogenicity; risk factor',
          'Uncertain significance; Affects',
          'Uncertain significance; association',
          'Uncertain significance; drug response',
          'Uncertain significance; other',
          'Uncertain significance; risk factor',
          'association',
          'confers sensitivity',
          'drug response',
          'drug response; other',
          'drug response; risk factor',
          'no interpretation for the single variant',
          'not provided',
          'other',
          'protective',
          'protective; risk factor'}

In [ ]:
import collections
num_data = 0
#faut-il que je le fasse plutôt comme ça pour le dico général?
pathogenicity = collections.defaultdict(dict)
for g in dic:
    num_patho = 0
    num_benign = 0
    num_vus = 0
    for a in dic[g]:
        wild_type = dic[g][a]['Change'][:3] 
        loc = int(dic[g][a]['Change'][3 : -3])
        #print(dic[g][a])
        #pas compris pourquoi ce if n'est pas solved par le filtre du dico
        if('ClinicalSignificance' in dic[g][a].keys()):
            num_data +=1
            if(dic[g][a]['ClinicalSignificance'] in pathogenic):
                num_patho += 1
                pathogenicity[g]['Num of Pathogenic'] = num_patho

            if(dic[g][a]['ClinicalSignificance'] in benign):
                num_benign += 1
                pathogenicity[g]['Num of Benign'] = num_benign

            if(dic[g][a]['ClinicalSignificance'] in vus): 
                num_vus += 1
                pathogenicity[g]['Num of VUS'] = num_vus
            if(num_patho == 0):
                   pathogenicity[g]['Num of Pathogenic'] = num_patho
            if(num_benign == 0):
                pathogenicity[g]['Num of Benign'] = num_benign
            if(num_vus == 0):
                pathogenicity[g]['Num of VUS'] = num_vus
                
print("number of filtered data", num_data)


In [ ]:
print(pathogenicity)

## Visualisation

In [ ]:

print(sum(pathogenicity[ge]['Num of VUS'] for ge in pathogenicity), "VUSs")
print(sum(pathogenicity[ge]['Num of Pathogenic'] for ge in pathogenicity), 'Pathogenic')
print(sum(pathogenicity[ge]['Num of Benign'] for ge in pathogenicity), 'Benign')

X = np.arange(len(pathogenicity))
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.set_title('Distribution of Pathogenicity per gene')
ax.set_ylabel('Number of cases')

ax.bar(X + 0.00, [pathogenicity[ge]['Num of VUS'] for ge in pathogenicity], color ='#fcff42', width = 0.25)
ax.bar(X + 0.25, [pathogenicity[ge]['Num of Pathogenic'] for ge in pathogenicity], color = '#426eff', width = 0.25)
ax.bar(X + 0.50, [pathogenicity[ge]['Num of Benign'] for ge in pathogenicity], color = 'r', width = 0.25)
plt.legend(['VUS','Benign', 'Pathogenic'])   
    
plt.show()

## PREPROCESS FOR LSTM

### Padding sequences to feed them to model 

In [ ]:
ordered_daatol = [daatol[aa] for aa in daatol]
ordered_daatol.sort()
x_alphabet = {}
x_alphabet['<PAD>'] = 0
i = 1
for aa in ordered_daatol:
    x_alphabet[aa] = i
    i += 1

In [ ]:
y_alphabet = {'Benign' : 0, 'Pathogenic' : 1}

In [ ]:
mut_seq_dic = {gene : {} for gene in dic}

In [ ]:
shifted_seq_dic = {}

## Create a dictionnary with the mutated sequence and mutation centered if length > 1022

In [ ]:
max_length = 1022
for g in dic:
    for a in dic[g]:
        mutation = dic[g][a]['Change'][-3:] 
        loc = int(dic[g][a]['Change'][3 : -3])
        if (mutation != 'dup'):
            if ('ClinicalSignificance' in dic[g][a]):
                if(dic[g][a]['ClinicalSignificance'] not in vus):
                    aa = daatol[mutation]
                    l_mutated_seq = list(seq_dic[g])
                    l_mutated_seq[loc - 1] = aa
                    if(loc > 1023):
                        shifted_seq_dic[g] = seq_dic[g][max(0, loc - 1 - max_length // 2) : min(loc - 1 + max_length // 2, len(l_mutated_seq))]
                        l_mutated_seq = l_mutated_seq[max(0, loc - 1 - max_length // 2) : min(loc - 1 + max_length // 2, len(l_mutated_seq))]
                    else:
                        shifted_seq_dic[g] = seq_dic[g][:max_length]
                        l_mutated_seq = l_mutated_seq[:max_length]
                    mutated_seq = "".join([str(i) for i in l_mutated_seq])
                    mut_seq_dic[g][a] = mutated_seq

# The Model

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from sklearn import metrics

In [ ]:
import esm

In [ ]:
class AaSeqClassifier(nn.Module):
    def __init__(self):
        super(AaSeqClassifier, self).__init__()
         # IMPROTANT: Path a un folder de ton choix sur ton ordi
            
        torch.hub.set_dir("/data/folder")

        # Load ESM-2 model
        model, alphabet = esm.pretrained.esm2_t30_150M_UR50D()
        self.padID = alphabet.padding_idx
        self.model = model 
        # Extract per-residue representations (on CPU)

        # Generate per-sequence representations via averaging
        # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1
        self.fc1 = nn.Linear(in_features= 640,
                             out_features= 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x, x_true, cuda: bool = False):
        #with torch.no_grad():
        pad_mask = x != self.padID
        results = model(x, repr_layers=[30], return_contacts=False)
        results_true = model(x_true, repr_layers=[30], return_contacts=False)
        
        token_representations = results["representations"][30]
        token_representations_true = results_true["representations"][30]
        
        out = token_representations - token_representations_true
        
        out = (out * pad_mask.unsqueeze(2)).sum(dim = 1) / pad_mask.sum(dim= 1, keepdim = True)
        out = self.fc2(self.fc1(out))
        return out

## FOR Language Model

### Data Processing ESM-2

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class DatasetMaper(Dataset):
    '''Handles batches of dataset'''
  
    def __init__(self, x, x_true, y):
        self.x = x
        self.x_true = x_true
        self.y = y

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.x_true[idx], self.y[idx]

## Training 

In [ ]:
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score

In [ ]:
#metrics.accuracy_score(y_pred.cpu().detach().numpy(), y.cpu().detach().numpy())), end='')

In [ ]:
#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    correct = (preds == y)
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train_model(model, optimizer, dataloader_train, dataloader_test, epochs):
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0

  # model in training mode
    model.train()

    #x_batch, y_batch = next(iter(loader_training))
    for x_batch, x_true_batch, y_batch in tqdm(loader_training):

        #resets the gradients after every batch
        optimizer.zero_grad()
        
        x = torch.tensor(x_batch).to(torch.long)
        x_true = torch.tensor(x_true_batch).to(torch.long)
        y = torch.tensor(y_batch).to(torch.long)
        lengths = torch.tensor(lengths).to(torch.long)

        if torch.cuda.is_available():
            x = x.cuda()
            x_true = x_true.cuda()
            y = y.cuda()
            lengths = lengths.cuda()

        # Feed the model and get output "y_pred"
        y_pred = model(x, x_true)

        # Calculate loss
        loss = F.cross_entropy(y_pred, y)
        
        #compute the binary accuracy
        #acc = binary_accuracy(torch.argmax(y_pred, dim=1), y)

        # The gradients are calculated
        #backpropage the loss and compute the gradients
        # i.e. derivatives are calculated
        loss.backward()

        # Each parameter is updated
        # with torch.no_grad():
        #     a -= lr * a.grad
        #     b -= lr * b.grad
        optimizer.step()
        
        #loss and accuracy
        epoch_loss += loss.item()  
        #epoch_acc += acc.item()    
        
    return epoch_loss / len(loader_training)#, epoch_acc / len(loader_training)
        
#        print('\rEp {}/{}, loss train: {:.2f}, 
#                  format(epoch + 1, epochs, len(loader_training), loss)

In [ ]:
def evaluation():

    predictions = []
    y_true = []
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0
    epoch_auc = 0
    #model is turned in evaluation mode
    model.eval()

    # Skipping gradients update
    #Disabling gradient calculation is useful for inference,
    #when you are sure that you will not call Tensor.backward().
    #It will reduce memory consumption for computations that would otherwise have requires_grad=True
    with torch.no_grad():

        for x_batch, x_true_batch, y_batch in tqdm(loader_valid):
        
            x = torch.tensor(x_batch).to(torch.long)
            x_true = torch.tensor(x_true_batch).to(torch.long)
            y = torch.tensor(y_batch).to(torch.long)
            
            if torch.cuda.is_available():
                x = x.cuda()
                x_true = x_true.cuda()
                y = y.cuda()

        
            y_pred = model(x, x_true)
        
            predictions += list(y_pred.detach().cpu().numpy())
            y_true += list(y.detach().cpu().numpy())
            
            loss = F.cross_entropy(y_pred, y)
            acc = binary_accuracy(torch.argmax(y_pred, dim = 1).detach().cpu().numpy(), y.detach().cpu().numpy())
            
            #keep track of loss and accuracy
            epoch_loss += loss.cpu().item()
            epoch_acc += acc.item()
            
    auc = roc_auc_score(torch.FloatTensor(y_true).numpy(), torch.softmax(torch.FloatTensor(predictions), dim=1)[:, 1])
    return epoch_loss / len(loader_valid), epoch_acc / len(loader_valid), auc

## ------------------------------------------------------------

In [ ]:
import torch

In [ ]:
import os
path= os.path.expanduser('~/your_folder')

# IMPROTANT: Path a un folder de ton choix sur ton ordi
torch.hub.set_dir("/your_folder")

# Load ESM-2 model
_, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()

#if torch.cuda.is_available():
#    model = model.cuda()
#    print("Transferred model to GPU")

data_esm2 = []
for g in mut_seq_dic:
    for a in mut_seq_dic[g]:
        data_esm2.append((f"{g}_{a}_wt", shifted_seq_dic[g]))
        data_esm2.append((f"{g}_{a}_mut", mut_seq_dic[g][a]))  
        
batch_labels, _, batch_tokens = batch_converter(data_esm2)

x_data, y_data, x_true_data = {gene : {} for gene in dic}, {gene : {} for gene in dic}, {gene : {} for gene in dic}
for label, token in zip(batch_labels, batch_tokens):
        g = label.split('_')[0]
        a =  int(label.split('_')[1])
        typ =  label.split('_')[2]
        if (typ == 'wt'):
            x_true_data[g][a] = token
        else: 
            x_data[g][a] = token
            
        if dic[g][a]['ClinicalSignificance'] in benign:
            y_data[g][a] = y_alphabet['Benign']
        if dic[g][a]['ClinicalSignificance'] in pathogenic:        
            y_data[g][a] = y_alphabet['Pathogenic']
            
#----------------------------Split train test valid--------------------------------------------------------------------
import json
  
# Iterating through the json
# list
x_train , x_true_train, x_valid, x_true_valid, x_test, x_true_test, y_train, y_valid, y_test = [],[],[],[],[],[],[],[],[]
# Opening JSON file
 f = open(os.path.expanduser("~/MIT/split.json"))
# returns JSON object as 
# a dictionary
data = json.load(f)
for g in x_data:
    for a in x_data[g]:
        if (data[g] == 'train'):
            x_train.append(x_data[g][a])
            x_true_train.append(x_true_data[g])
            y_train.append(y_data[g][a])
        if (data[g] == 'dev'):
            x_valid.append(x_data[g][a])
            x_true_valid.append(x_true_data[g])
            y_valid.append(y_data[g][a])
        else:
            x_test.append(x_data[g][a])
            x_true_test.append(x_true_data[g])
            y_test.append(y_data[g][a])

# Closing file
f.close()
#---------------------------------------------------------------------------------------------------------------

## Pad Sequences for train, valid, test

In [ ]:
#-------------------------------Padding sequences---------------------------------------------------------------
X_lengths = [len(sequence) for sequence in x_train]
# create an empty matrix with padding tokens
pad_token = x_alphabet['<PAD>']
longest_seq = max_length

#---------X-TRAIN---------------
padded_X_train = np.ones((len(x_train), longest_seq)) * pad_token
#copy over the actual sequences
for i, x_len in enumerate(X_lengths):
    sequence = x_train[i]
    padded_X_train[i, 0:x_len]= sequence[:x_len]
#--------X-VALID-----------------
X_lengths_valid = [len(sequence) for sequence in x_valid]
# create an empty matrix with padding tokens
padded_X_valid = np.ones((len(x_valid), longest_seq)) * pad_token
#copy over the actual sequences
for i, x_len in enumerate(X_lengths_valid):
        sequence = x_valid[i]
        padded_X_valid[i, 0:x_len] = sequence[:x_len]

#--------X-TEST-------------------
X_lengths_test = [len(sequence) for sequence in x_test]
# create an empty matrix with padding tokens
padded_X_test = np.ones((len(x_test), longest_seq)) * pad_token
#copy over the actual sequences
for i, x_len in enumerate(X_lengths_test):
    sequence = x_test[i]
    padded_X_test[i, 0:x_len] = sequence[:x_len]

#--------------X_TRUE-----------
X_true_lengths =  [len(sequence) for sequence in x_true_train]
padded_X_true_train = np.ones((len(x_true_train), longest_seq)) * pad_token
for i, x_len in enumerate(X_true_lengths):
    sequence = x_true_train[i]
    padded_X_true_train[i, 0:x_len] = sequence[:x_len]
#---------
X_true_lengths_valid = [len(sequence) for sequence in x_true_valid]
# create an empty matrix with padding tokens
padded_X_true_valid = np.ones((len(x_true_valid), longest_seq)) * pad_token
#copy over the actual sequences
for i, x_len in enumerate(X_true_lengths_valid):
        sequence = x_true_valid[i]
        padded_X_true_valid[i, 0:x_len] = sequence[:x_len]
#--------------------
X_true_lengths_test = [len(sequence) for sequence in x_true_test]
# create an empty matrix with padding tokens
padded_X_true_test = np.ones((len(x_true_test), longest_seq)) * pad_token
#copy over the actual sequences
for i, x_len in enumerate(X_true_lengths_test):
        sequence = x_true_test[i]
        padded_X_true_test[i, 0:x_len] = sequence[:x_len]
#------------------------------------------------------

In [ ]:
epochs = 20
model = AaSeqClassifier()

if torch.cuda.is_available():
    model = model.cuda()
    print("Transferred model to GPU")

train_batch_size = 32
batch_size = 32
training_set = DatasetMaper(x_train, x_true_train, y_train)
validation_set = DatasetMaper(x_valid, x_true_valid, y_valid)
test_set = DatasetMaper(x_test, x_true_test, y_test)
loader_training = DataLoader(training_set, batch_size = train_batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(validation_set, batch_size = batch_size)

loader_test = DataLoader(test_set, batch_size = batch_size)
        
    
optimizer = optim.Adam(model.parameters(), lr = 1e-4)
#print(model)
#print(f"The model has {sum([x.nelement() for x in model.parameters()]):,} parameters.")
best_valid_loss = float('inf')
criterion = nn.CrossEntropyLoss()
for epoch in tqdm(range(epochs)):
     
    #train the model
    train_loss = train_model(model, optimizer, loader_training, criterion, epochs)
    
    #evaluate the model
    valid_loss, valid_acc, valid_auc = evaluation()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), path)
    
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | Val. AUC: {valid_auc*100:.2f}')
    
    #Erreur : Expected all tensors to be on the same device, but found at least two devices,
    #cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)

model.load_state_dict(torch.load(path));
model.eval();


In [ ]:
train_loss

# plot

In [ ]:
import esm

In [ ]:

# Look at the unsupervised self-attention map contact predictions
import matplotlib.pyplot as plt
for (_, seq), attention_contacts in zip(data_esm2, results["contacts"]):
    plt.matshow(attention_contacts[: len(seq), : len(seq)])
    plt.title(seq)
    plt.show()





In [ ]:
if __name__ == "__main__":
    results_esm2 = main()

In [ ]:
from sklearn.metrics import roc_auc_score

global_scores = [torch.sigmoid(torch.tensor(results_esm_1v[g][a])).item() for g in results_esm_1v for a in results_esm_1v[g]]
global_labels = [dic[g][a]["ClinicalSignificance"] in benign for g in results_esm_1v for a in results_esm_1v[g]]

print(roc_auc_score(global_labels, global_scores))